In [1]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, time
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose


install_package("fastai")
# install_package("fastai==1.0.61")
import fastai
from fastai.tabular.core import add_datepart
from fastai.imports import *


%matplotlib inline


import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


# INTRO STUFF

In [3]:
df_baseline_15m = pd.read_csv('final_data_15min.csv')
df_baseline_1d = pd.read_csv('final_data_day.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'final_data_15min.csv'

In [ ]:
#in case, faster than re-reading the csv
df_15m = df_baseline_15m.copy(deep = True)
df_1d = df_baseline_1d.copy(deep = True)

In [ ]:
#Quick explo

# df_15m.head(5)
# df_15m.describe()
# df_15m.info()
# df_15m.isnull().sum()
# unique_15m = {}
# for c in df_15m.columns:
#   unique_15m[c] = list(df_15m[str(c)].unique())
# for key, value in unique_15m.items():
#   print(f"{key}: {value[::-1][:10]}")
'''
Some 'nan' issues persist : WEATHER MAIN , TEMP MAX, etc...
'''

#Quick explo 2
# df_1d.head(5)
# df_1d.describe()
# df_1d.info()
# df_1d.isnull().sum()
unique_1d = {}
for c in df_1d.columns:
  unique_1d[c] = list(df_15m[str(c)].unique())
for key, value in unique_1d.items():
  print(f"{key}: {value[::-1][:10]}")
# print(unique_1d)


DATE: ['2022-08-18', '2022-08-17', '2022-08-16', '2022-08-15', '2022-08-14', '2022-08-13', '2022-08-12', '2022-08-11', '2022-08-10', '2022-08-09']
ATTRACTION NAME: ['Vertical Drop', 'Circus Train', 'Giga Coaster', 'Crazy Dance', 'Inverted Coaster', 'Drop Tower', 'Superman Ride', 'Haunted House', 'Zipline', 'Giant Wheel']
WAIT TIME: [245, 240, 210, 250, 190, 215, 185, 170, 165, 225]
NB UNITS: [19.1333, 106.8, 124.2, 95.6667, 121.067, 122.067, 21.1333, 70.6667, 96.4667, 97.4667]
GUEST CARRIED: [80.3335, 42.0004, 127.8, 397.5, 686.0, 122.334, 189.333, 195.001, 66.2662, 61.6665]
CAPACITY: [383.834, 246.75, 191.917, 82.2501, 137.083, 274.167, 27.4167, 164.5, 219.334, 356.417]
ADJUST CAPACITY: [602.7, 353.683, 345.483, 226.3, 267.333, 150.883, 198.783, 137.167, 54.8333, 246.75]
OPEN_TIME: [12, 11, 14, 2, 10, 7, 13, 5, 1, 8]
REAL TIME ATTRACTION OPEN FOR 15MIN: [-7, -3, -12, -8, -6, -9, -5, -4, -10, -13]
REAL TIME ATTRACTION CLOSES FOR 15MIN: [30, 7, 5, 14, 2, 9, 10, 13, 11, 12]
NB MAX UNITS:

### *Util Functions*

In [ ]:
#Get seasonality

# def get_season(date):
#     year = int(date[0:4])
#     seasons = {
#         'spring': pd.date_range(start=f"20/03/{year}", end=f"20/06/{year}"),
#         'summer': pd.date_range(start=f"21/06/{year}", end=f"22/09/{year}"),
#         'autumn': pd.date_range(start=f"23/09/{year}", end=f"20/12/{year}"),
#         'winter': pd.date_range(start=f"21/12/{year}", end=f"19/03/{year + 1}"),
#     } #standard starts of each season

#     for season, date_range in seasons.items():
#         if date in date_range:
#             return season

#     return 'winter'  # Default to winter if date is not found in the ranges above


# def get_time_period(time_vals):
#     time_str = str(time_vals[-6:-1])
#     input_time = datetime.strptime(time_str, '%H:%M').time()

#     # Define time periods
#     morning_start = time(9, 0)
#     morning_end = time(12, 59)
#     afternoon_start = time(13, 0)
#     afternoon_end = time(18, 29)
#     evening_start = time(18, 30)
#     evening_end = time(23, 59)
#     night_end = time(1, 0)  # for times after midnight up to 1:00 AM

#     # Determine and return the time period
#     if morning_start <= input_time <= morning_end:
#         return 'morning_period'
#     elif afternoon_start <= input_time <= afternoon_end:
#         return 'afternoon_period'
#     elif evening_start <= input_time <= evening_end or input_time < night_end:
#         return 'evening_period'
#     else:
#         # This handles the case from midnight to 1:00 AM as part of the evening period
#         return 'evening_period'





In [ ]:
#more efficient version of the commented-out code in the prev block:

def get_season(date):
    month = date.month
    day = date.day
    if (month > 3 and month < 6) or (month == 3 and day >= 20) or (month == 6 and day < 21):
        return 'spring'
    elif (month > 6 and month < 9) or (month == 6 and day >= 21) or (month == 9 and day < 23):
        return 'summer'
    elif (month > 9 and month < 12) or (month == 9 and day >= 23) or (month == 12 and day < 21):
        return 'autumn'
    else:
        return 'winter'


def get_time_period(time_vals):
    hours, minutes = map(int, time_vals[-8:-3].split(':'))

    if 9 <= hours < 13:
        return 'morning'
    elif 13 <= hours < 18 or (hours == 18 and minutes < 30):
        return 'afternoon'
    else:
        return 'evening'

def robustization(data, col_name):
    data_mean = data[col_name].mean()
    data_stdev = data[col_name].std()

    # Bounds
    lower_bound = data_mean - 4.5 * data_stdev
    upper_bound = data_mean + 4.5 * data_stdev

    # Filtering
    filtered_data = data[(data[col_name] >= lower_bound) & (data[col_name] <= upper_bound)]
    filtered_data = filtered_data[col_name].apply(lambda x : np.sqrt(x))

    return filtered_data

In [ ]:
#Util functions to correct stationarity:

def differencing(df, col_name, second = False):

  diff_1 = col_name + 'diff'
  df[diff_1] = df[col_name].diff()

  if second:
    diff_2 = diff_1 + 'diff'
    df[diff_2] = df[diff_1].diff()
    df.drop(columns = [diff_1], axis = 1, inplace = True)

  df.dropna(inplace = True)

  return df


def log_transfo(df, col_name, differenting = False):

  # df[col_name].apply(lambda x : np.log(x))
  col_log = col_name + "log"

  df[col_log] = np.log(df[col_name])

  if differenting:
    col_log_diff = col_log + "diff"
    df[col_log_diff] = df[col_log].diff().dropna()

  return df


def sqrt_transfo(df, col_name, differenting = False):

  col_sqrt = col_name + "sqrt"

  df[col_sqrt] = np.sqrt(df[col_name])

  if differenting:
    col_sqrt_diff = col_sqrt + "diff"
    df[col_sqrt_diff] = df[col_sqrt].diff().dropna()

  return df


def remove_seasonality(df, col_name, model = 'additive', freq = None):
  '''
  for freq : e.g. freq = 12 --> monthly data
  '''

  if freq is None:
    freq = pd.infer_freq(df.index) #automatically gives frequency

  decomposition = seasonal_decompose(df[col_name], model = model, period = freq)

  if model == 'additive':
    deseasonalized = df[col_name] - decomposition.seasonal

  else: #mutliplicative
    deseasonalized = df[col_name] / decomposition.seasonal

  return deseasonalized


In [ ]:
# df_15m['START OF 1 RIDE FOR ATTRACTION'] = df_15m['START OF 1 RIDE FOR ATTRACTION'].dt.strftime("%Y-%m-%d %H:%M")
# print(df_15m['START OF 1 RIDE FOR ATTRACTION'][1][-8:-3])

09:00


In [ ]:
# Apply the function to create the 'seasonality' column

from tqdm import tqdm
tqdm.pandas()


df_15m['DATE'] = pd.to_datetime(df_15m['DATE'])
# df_15m['START OF 1 RIDE FOR ATTRACTION'] = df_15m['START OF 1 RIDE FOR ATTRACTION'].dt.strftime("%Y-%m-%d %H:%M") #already a string
df_1d['DATE'] = pd.to_datetime(df_1d['DATE'])


print("Building Seasonality")
print("---")
df_15m['SEASONALITY'] = df_15m['DATE'].progress_apply(get_season)

print("---")
df_1d['SEASONALITY'] = df_1d['DATE'].progress_apply(get_season)

print()

print("Building Day Periodicity")
print("---")
df_15m['DAY PERIOD'] = df_15m['START OF 1 RIDE FOR ATTRACTION'].progress_apply(get_time_period)
print("---")


# df_15m.head(5)
# df_15m.tail(5)
# df_1d.head(5)
# df_1d.tail(5)

Building Seasonality
---


100%|██████████| 2369821/2369821 [00:09<00:00, 246029.14it/s]


---


100%|██████████| 42340/42340 [00:00<00:00, 392312.30it/s]



Building Day Periodicity
---


100%|██████████| 2369821/2369821 [00:04<00:00, 587145.18it/s]

---


In [ ]:
# df_1d[df_1d['SEASONALITY']=='summer']

len(df_15m['SEASONALITY'])
len(df_1d['SEASONALITY'])


42340

In [ ]:
#Plotting with Custom Bins

def plot_temperature_distribution(df, col_name, precision, xlabel, ylabel, title):
    '''
    When trying to plot distributions of numerical data with high floating point precision,
    Use this function to simplify the rendering.
    For e.g., for Temperature felt, Precision = 1 gives 1º accuracies.

    ----------
    Parameters:
    - df : df used
    - col_name : target column name
    - precision : bin precision (1 for 1ºC, 0.5 for 0.5ºC, etc...). Must be +ve float
    - xlabel, ylabel, title : for plotting

    '''

    #Error Handling
    if not (isinstance(precision, float) and precision > 0):
      raise ValueError("Precision must be a positive floating number.")

    # Categorize data into bins
    df_temp = pd.DataFrame()
    df_temp[col_name] = df[col_name].apply(lambda x: np.round(x / precision) * precision)

    # Plot
    plt.figure(figsize=(10, 6))
    sns.countplot(x= col_name, data=df_temp)
    plt.title(f'{title} ({precision}-degree bins)')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


# BASIC EXPLORATION

In [ ]:
#Basic explo

# Weather Condition Distribution
sns.countplot(x='WEATHER MAIN', data=df_15m)
plt.title('Weather Condition Distribution')
plt.xticks(rotation=45)
plt.show()
print()

# Seasonality Distribution
sns.countplot(x='SEASONALITY', data=df_15m)
plt.title('Seasonality Distribution')
plt.xticks(rotation=45)
plt.show()
print()

# Temperature Felt Distribution
precision = 1
xlabel = 'Temperatures Felt'
ylabel = 'Count'
title = 'Histogram of Felt Temperatures'
plot_temperature_distribution(df_15m, 'TEMPERATURE FELT', float(precision), xlabel, ylabel, title)

# TRANSFORMATIONS

Here feature engineering for our collected data.
Feel free to add/change/etc...

In [ ]:
#Deep copy to prevent mutable datatype issues
df_15m_transf = df_15m.copy(deep = True)
df_1d_transf = df_1d.copy(deep = True)

df_15m_transf.head(5)

,DATE,START OF 1 RIDE FOR ATTRACTION,HOUR START OF 1 RIDE,END OF 1 RIDE FOR ATTRACTION,ATTRACTION NAME,WAIT TIME,NB UNITS,GUEST CARRIED,CAPACITY,ADJUST CAPACITY,...,WIND DEG,CLOUD PERCENTAGE,WEATHER MAIN,ATTRACTION,ATTENDANCE,IS NIGHT SHOW,IS PARADE 1,IS PARADE 2,SEASONALITY,DAY PERIOD
0,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Rapids Ride,0,2.0,0.0,0.000,0.00,...,196.0,100.0,Clouds,Rapids Ride,0.0,0.0,0.0,0.0,winter,morning
1,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Merry Go Round,0,65.0,0.0,0.000,0.00,...,196.0,100.0,Clouds,Merry Go Round,0.0,0.0,0.0,0.0,winter,morning
2,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Bumper Cars,5,18.0,108.0,254.749,254.75,...,196.0,100.0,Clouds,Bumper Cars,0.0,0.0,0.0,0.0,winter,morning
3,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Go-Karts,0,2.0,0.0,0.000,0.00,...,196.0,100.0,Clouds,Go-Karts,0.0,0.0,0.0,0.0,winter,morning
4,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Spinning Coaster,0,1.4,0.0,315.750,52.65,...,196.0,100.0,Clouds,Spinning Coaster,0.0,0.0,0.0,0.0,winter,morning


### Changing the Precision of some floating points

In [ ]:
precision = 1

# lst_cols = ['TEMPERATURE', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM', 'WIND SPEED']

# print("Changing Precision of Floating Points")
# for i in tqdm(lst_cols):
#   df_15m_transf[i] = df_15m_transf[i].apply(lambda x: np.round(x / precision) * precision)
#   df_1d_transf[i] = df_1d_transf[i].apply(lambda x: np.round(x / precision) * precision)
# only needed for plotting.


### ENCODING CATEGORICAL VARIABLES

In [ ]:
df_15m_transf['ATTRACTION NAME'].unique()
df_15m_transf['ATTRACTION'].unique()

print('df_1d')
print(df_1d_transf.columns)
print()
print('df_15m')
print(df_15m_transf.columns)

df_1d
Index(['DATE', 'ATTRACTION NAME', 'WAIT TIME', 'NB UNITS', 'GUEST CARRIED',
       'CAPACITY', 'ADJUST CAPACITY', 'OPEN_TIME',
       'REAL TIME ATTRACTION OPEN FOR 15MIN',
       'REAL TIME ATTRACTION CLOSES FOR 15MIN', 'NB MAX UNITS', 'TEMPERATURE',
       'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
       'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE',
       'ATTENDANCE', 'SEASONALITY'],
      dtype='object')

df_15m
Index(['DATE', 'START OF 1 RIDE FOR ATTRACTION', 'HOUR START OF 1 RIDE',
       'END OF 1 RIDE FOR ATTRACTION', 'ATTRACTION NAME', 'WAIT TIME',
       'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'OPEN_TIME',
       'REAL TIME ATTRACTION OPEN FOR 15MIN',
       'REAL TIME ATTRACTION CLOSES FOR 15MIN', 'NB MAX UNITS', 'TEMPERATURE',
       'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
       'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE',
       'WEATHER MAIN', 'AT

In [ ]:
assert (df_15m_transf['ATTRACTION NAME'] == df_15m_transf['ATTRACTION']).all()
df_15m_transf.drop(['ATTRACTION'], axis = 1, inplace = True)



In [ ]:
assert sorted(df_15m_transf['ATTRACTION NAME'].unique()) == sorted(df_1d_transf['ATTRACTION NAME'].unique())

In [ ]:
'''
The column 'WEATHER MAIN' does not exist in df_1d, so in the following I add it: (from df_15m):
'''

# Creating a temp df and transfering target columns to it
df_temp = pd.DataFrame()
df_temp[['DATE', 'WEATHER MAIN']] = df_15m_transf[['DATE', 'WEATHER MAIN']]

# Building a set to only keep unique column value-couples :
# unique_rows = df_temp.loc[~df_temp.duplicated(subset=['DATE', 'WEATHER MAIN'], keep=False)] - did not work
temp_set = set()
for i in range(len(df_temp['DATE'])):
  temp_set.add(tuple((df_temp['DATE'][i], df_temp['WEATHER MAIN'][i])))


#MAPPING SET VALUES TO THE TARGET DATES IN df_1d_transf['WEATHER MAIN']

temp_lst = list(temp_set)

# SLOW WAY (nested loops):
# temp_dic = dict(temp_lst)
# df_1d_transf['WEATHER MAIN'] = [0] * len(df_1d_transf['DATE'])
# for i in tqdm(range(len(df_1d_transf['DATE']))):
#   for j in temp_lst:
#     if df_1d_transf['DATE'].iloc[i] == j[0]:
#        df_1d_transf['WEATHER MAIN'].iloc[i] = j[1]

# FASTER WAY (vectorized pd operations))
date_weather_dict = dict(temp_lst)
df_1d_transf['WEATHER MAIN'] = df_1d_transf['DATE'].map(date_weather_dict).fillna(0)
# print(df_1d_transf['WEATHER MAIN'])
# print(df_1d_transf.columns)

In [ ]:
#Proceeding with the label encoder - DUMMY VARIABLES



#Correcting issue with 'WEATHER MAIN': there is a numerical datatype that needs to be textual for categ var encoding
df_15m_transf['WEATHER MAIN'] = df_15m_transf['WEATHER MAIN'].apply(lambda x: 'num_val' if str(x) == '70.63771082719657' else x)
df_1d_transf['WEATHER MAIN'] = df_1d_transf['WEATHER MAIN'].apply(lambda x: 'num_val' if str(x) == '70.63771082719657' else x)


print("Encoding Categorical Variables with Dummies")
lst_cols_categ = ['ATTRACTION NAME', 'WEATHER MAIN', 'SEASONALITY', 'DAY PERIOD']
for c in tqdm(lst_cols_categ):
  dummies_15m = pd.get_dummies(df_15m_transf[c], prefix = c)
  df_15m_transf = df_15m_transf.drop(c, axis = 1)
  df_15m_transf = pd.concat([df_15m_transf, dummies_15m], axis = 1)

lst_cols_categ = ['ATTRACTION NAME', 'WEATHER MAIN', 'SEASONALITY']
for c in tqdm(lst_cols_categ):
  dummies_1d = pd.get_dummies(df_1d_transf[c], prefix = c)
  df_1d_transf = df_1d_transf.drop(c, axis = 1)
  df_1d_transf = pd.concat([df_1d_transf, dummies_1d], axis = 1)


print("Adding Information on DATE with fastai package")
add_datepart(df_1d_transf, 'DATE')
add_datepart(df_15m_transf, 'DATE')


Encoding Categorical Variables with Dummies


100%|██████████| 3/3 [00:00<00:00, 51.96it/s]


Adding Information on DATE with fastai package


,START OF 1 RIDE FOR ATTRACTION,HOUR START OF 1 RIDE,END OF 1 RIDE FOR ATTRACTION,WAIT TIME,NB UNITS,GUEST CARRIED,CAPACITY,ADJUST CAPACITY,OPEN_TIME,REAL TIME ATTRACTION OPEN FOR 15MIN,...,DATEDay,DATEDayofweek,DATEDayofyear,DATEIs_month_end,DATEIs_month_start,DATEIs_quarter_end,DATEIs_quarter_start,DATEIs_year_end,DATEIs_year_start,DATEElapsed
0,2018-01-01 09:00:00,9,2018-01-01 09:15:00,0,2.0,0.0,0.000,0.00,0,0,...,1,0,1,False,True,False,True,False,True,1.514765e+09
1,2018-01-01 09:00:00,9,2018-01-01 09:15:00,0,65.0,0.0,0.000,0.00,0,0,...,1,0,1,False,True,False,True,False,True,1.514765e+09
2,2018-01-01 09:00:00,9,2018-01-01 09:15:00,5,18.0,108.0,254.749,254.75,15,15,...,1,0,1,False,True,False,True,False,True,1.514765e+09
3,2018-01-01 09:00:00,9,2018-01-01 09:15:00,0,2.0,0.0,0.000,0.00,0,0,...,1,0,1,False,True,False,True,False,True,1.514765e+09
4,2018-01-01 09:00:00,9,2018-01-01 09:15:00,0,1.4,0.0,315.750,52.65,9,9,...,1,0,1,False,True,False,True,False,True,1.514765e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369816,2022-08-18 22:45:00,22,2022-08-18 23:00:00,0,1.0,0.0,0.000,0.00,0,0,...,18,3,230,False,False,False,False,False,False,1.660781e+09
2369817,2022-08-18 22:45:00,22,2022-08-18 23:00:00,10,5.0,252.0,503.750,503.75,15,15,...,18,3,230,False,False,False,False,False,False,1.660781e+09
2369818,2022-08-18 22:45:00,22,2022-08-18 23:00:00,0,2.0,0.0,0.000,0.00,0,0,...,18,3,230,False,False,False,False,False,False,1.660781e+09
2369819,2022-08-18 22:45:00,22,2022-08-18 23:00:00,0,9.0,0.0,0.000,0.00,0,0,...,18,3,230,False,False,False,False,False,False,1.660781e+09


In [ ]:
cols_to_drop = ["DATEDayofyear", "DATEElapsed", "HOUR START OF 1 RIDE	"]

for c in cols_to_drop:
  df_1d_transf.drop(c, axis = 1, inplace = True)
  df_15m_transf.drop(c, axis = 1, inplace = True)

KeyError: "['HOUR START OF 1 RIDE'] not found in axis"

In [ ]:
df_1d_transf.head(5)

In [ ]:
df_15m_transf.head(5)

### STANDARDIZATION (Z-Normalization)

In [ ]:
print(df_15m_transf.columns)
print(df_1d_transf.columns)

In [ ]:

#Min Max Scaling
minmax_scaler = MinMaxScaler() #perhaps needed ? (if we implement a neural network)

#Z-Scaling
# z_scaler = StandardScaler() - can't use this with 1 dim data. Must array.reshape(1, -1)
lst_cols_rescale = ['WAIT TIME', 'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'NB MAX UNITS',
                    'TEMPERATURE', 'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
                    'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE', 'ATTENDANCE']

print("Standardizing Numerical Variables")
for i in tqdm(lst_cols_rescale):
  # checker = None
  try :
    # checker = '15m'
    temp_mean_15m = df_15m_transf[i].mean()
    temp_std_15m = df_15m_transf[i].std()
    df_15m_transf[i].apply(lambda x: (x - temp_mean_15m) / temp_std_15m)

    # checker = '1d'
    temp_mean_1d = df_1d_transf[i].mean()
    temp_std_1d = df_1d_transf[i].std()
    df_1d_transf[i].apply(lambda x: (x - temp_mean_1d) / temp_std_1d)

  except KeyError as err:
    print()
    if err not in df_15m_transf.columns:
      print(f'The {err} column is not present in the 15m dataframe.')
    if err not in df_1d_transf.columns:
      print(f'The {err} column is not present in the 1d dataframe.')
    pass


  # df_15m_transf[i] = z_scaler.fit_transform(df_15m_transf[i])
  # df_1d_transf[i] = z_scaler.fit_transform(df_1d_transf[i])

### ROBUSTIZATION
Removing outliers & log transformation

In [ ]:
num_cols = ['WAIT TIME', 'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'NB MAX UNITS', 'CLOUD PERCENTAGE', 'ATTENDANCE']

for i in num_cols:
    robustization(df_1d_transf, i)
    robustization(df_15m_transf, i)



### TESTING FOR STATIONARITY

In [ ]:
'''
Non-stationarity indicates different statitics at different time.
Because most ML algos are built on statistics, if they differ as time goes,
generalizing/robustness/reliability may not be feasible

Let's test!
'''

# for ADF stationarity test
import statsmodels.tsa.stattools as sm


In [ ]:
# Selecting the continuous var data cols:
lst_cols_test = ['WAIT TIME', 'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'NB MAX UNITS',
                    'TEMPERATURE', 'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
                    'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE', 'ATTENDANCE']


#Lambda function that checks stationarity:
isStationary = lambda s, p : print(f'{s} is stationary: p-value{p * 100: .2f}%'
                                   if p < 0.05 else print(f'{s} is not stationary: p-value{p * 100: .2f}%'))
for i in tqdm(lst_cols_test):
  isStationary(i, sm.adfuller(df_15m_transf[i], maxlag = 1)[1])
  print()

In [ ]:
from google.colab import files


df_15m_transf.to_csv('df_15m_transf_final_data.csv', index=False)
files.download('df_15m_transf_final_data.csv')

df_1d_transf.to_csv('df_1d_transf_final_data.csv', index=False)
files.download('df_1d_transf_final_data.csv')

### TRANSFORMATION W/ GROUPED DATA BY ATTRACTION:

Above results look weird. Everything is stationary, which almost never happen.

A key issue is that I run that analysis over all attractions.
It would make more sense to create separate dataframes for each attraction, and then run our models.

From a business point of view, it still makes sense.
If one attraction is busy, it means the rest is busy as well.
Think of it from the pov of the customer : why would he/she wait if another attraction is free?
There could be collinearity emerging from this behavior.
That is, whatever we find relative to one attraction is common to all...minding the weather, perhaps?

Let's see !

In [ ]:
#Deep copy to prevent mutable datatype issues
df_15m_baseline_gp = df_15m.copy(deep = True)
df_1d_baseline_gp = df_1d.copy(deep = True)

attractions = df_1d['ATTRACTION NAME'].unique()
print(attractions)

In [ ]:

# grouped_dfs_15m = {k: v for k, v in df_15m_baseline.groupby(df_15m['ATTRACTION NAME'])}
# grouped_dfs_1d = {k: v for k, v in df_1d_baseline.groupby(df_1d['ATTRACTION NAME'])}

# dic_15m_gdfs = {}
# for i in attractions:
#   dic_15m_gdfs[i] = grouped_dfs_15m[i]

# dic_1d_gdfs = {}
# for i in attractions:
#   dic_1d_gdfs[i] = grouped_dfs_1d[i]

#Simpler would have been:

dic_15m_gdfs = {}
for i in attractions:

  dic_15m_gdfs[i] = df_15m_baseline_gp[df_15m_baseline_gp['ATTRACTION NAME'] == i]

dic_1d_gdfs = {}
for i in attractions:
  dic_1d_gdfs[i] = df_1d_baseline_gp[df_1d_baseline_gp['ATTRACTION NAME'] == i]

# print(dic_15m_gdfs)

In [ ]:
print(dic_15m_gdfs['Bumper Cars'])#['GUEST CARRIED'])

In [ ]:
#Adding a WEATHER MAIN column & correcting incorrect value types.
for i in attractions:
  dic_1d_gdfs[i]['WEATHER MAIN'] = dic_1d_gdfs[i]['DATE'].map(date_weather_dict).fillna(0)
  dic_15m_gdfs[i]['WEATHER MAIN'] = dic_15m_gdfs[i]['DATE'].map(date_weather_dict).fillna(0)

  dic_1d_gdfs[i]['WEATHER MAIN'] = dic_1d_gdfs[i]['WEATHER MAIN'].apply(lambda x: 'num_val'
                                                                        if str(x) == '70.63771082719657' else x)
  dic_15m_gdfs[i]['WEATHER MAIN'] = dic_15m_gdfs[i]['WEATHER MAIN'].apply(lambda x: 'num_val'
                                                                        if str(x) == '70.63771082719657' else x)
  # dic_15m_gdfs[i].drop('ATTRACTION NAME', axis = 1)
  # dic_1d_gdfs[i].drop('ATTRACTION NAME', axis = 1)

# print(dic_1d_gdfs['Bumper Cars']['WEATHER MAIN'].iloc[-1])

In [ ]:
print(dic_1d_gdfs['Bungee Jump'].columns)
print()
print(dic_15m_gdfs['Bungee Jump'].columns)

# for i in attractions:
#    dic_1d_gdfs[i].drop(columns = ['ATTRACTION NAME'], axis = 1, inplace = True)
#    dic_15m_gdfs[i].drop(columns = ['ATTRACTION'], axis = 1, inplace = True)

In [ ]:
#Encoding categ variables:

lst_cols_categ = ['WEATHER MAIN', 'SEASONALITY']

for i in tqdm(attractions):
  for c in lst_cols_categ:

    dummies_15m = pd.get_dummies(dic_15m_gdfs[i][c], prefix = c)
    dic_15m_gdfs[i] = dic_15m_gdfs[i].drop(c, axis = 1)
    dic_15m_gdfs[i] = pd.concat([dic_15m_gdfs[i], dummies_15m], axis = 1)


    dummies_1d = pd.get_dummies(dic_1d_gdfs[i][c], prefix = c)
    dic_1d_gdfs[i] = dic_1d_gdfs[i].drop(c, axis = 1)
    dic_1d_gdfs[i] = pd.concat([dic_1d_gdfs[i], dummies_1d], axis = 1)

In [ ]:
#Stationarity Testing:

# Selecting the continuous var data cols:
lst_cols_test = ['WAIT TIME', 'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'NB MAX UNITS',
                    'TEMPERATURE', 'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
                    'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE', 'ATTENDANCE']


#Lambda function that checks stationarity:
# isStationary = lambda s, p : print(f'{s} is stationary: p-value{p * 100: .2f}%'
#                                    if p < 0.05 else print(f'{s} is not stationary: p-value{p * 100: .2f}%'))


def isStationary(s, df):
    if df.nunique() > 1:  # Checks if there's more than one unique value
        p = sm.adfuller(df, maxlag=1)[1]
        if p < 0.05:
            # print(f'{s} is stationary: p-value{p * 100: .2f}%')
            return True
        else:
            # print(f'{s} is not stationary: p-value{p * 100: .2f}%')
            return False
    else:
        # print(f'{s} is constant: Skipping ADF test')
        return True


ns_dic_1d = {}

for a in attractions:
  for i in lst_cols_test:
    temp = dic_1d_gdfs[a][i]
    if not isStationary(i, temp):
      ns_dic_1d.setdefault(a, []).append(i)
      # print(f'Column Data {i}\t of {a.upper()}\t is not stationary')

# print()

ns_dic_15m = {}
for a in attractions:
  for i in lst_cols_test:
    temp = dic_1d_gdfs[a][i]
    if not isStationary(i, temp):
      ns_dic_15m.setdefault(a, []).append(i)
      # print(f'Column {i}\t of {a.upper()}\t is not stationary')

In [ ]:
for k, v in ns_dic_1d.items():
  print(k, v)

print()

for k, v in ns_dic_15m.items():
  print(k, v)

In [ ]:
#Z-Norm

lst_cols_rescale = ['WAIT TIME', 'NB UNITS', 'GUEST CARRIED', 'CAPACITY', 'ADJUST CAPACITY', 'NB MAX UNITS',
                    'TEMPERATURE', 'TEMPERATURE FELT', 'TEMPERATURE MINIMUM', 'TEMPERATURE MAXIMUM',
                    'PRESSURE', 'HUMIDITY', 'WIND SPEED', 'WIND DEG', 'CLOUD PERCENTAGE', 'ATTENDANCE']

print("Standardizing Numerical Variables")

counter = 0
for a in attractions:
  counter += 1
  print(f'Attraction number {counter} over {len(attractions)} : {a.upper()}')

  for i in tqdm(lst_cols_rescale):
    try :
      temp_mean_15m = dic_1d_gdfs[a][i].mean()
      temp_std_15m = dic_1d_gdfs[a][i].std()
      dic_1d_gdfs[a][i].apply(lambda x: (x - temp_mean_15m) / temp_std_15m)

      temp_mean_1d = dic_15m_gdfs[a][i].mean()
      temp_std_1d = dic_15m_gdfs[a][i].std()
      dic_15m_gdfs[a][i].apply(lambda x: (x - temp_mean_1d) / temp_std_1d)

    except KeyError as err:
      print()
      if err not in dic_15m_gdfs[a].columns:
        print(f'The {err} column is not present in the 15m dataframe.')
      if err not in dic_1d_gdfs[a].columns:
        print(f'The {err} column is not present in the 1d dataframe.')
      pass

  print()


In [ ]:
for a in attractions:
  for i in lst_cols_test:
    temp = dic_1d_gdfs[a][i]
    if not isStationary(i, temp):
      print(f'Column Data {i}\t of {a.upper()}\t is not stationary')

# print()

for a in attractions:
  for i in lst_cols_test:
    temp = dic_1d_gdfs[a][i]
    if not isStationary(i, temp):
      print(f'Column {i}\t of {a.upper()}\t is not stationary')

# EDA Part II : Better visuals

Let's try to automate some renderings.

For example, say you want to plat the waiting time per attraction, or waiting time per attraction per season,...

In [ ]:

def plot_waiting_times(df, attraction_name):

    df_attraction = df[df['ATTRACTION NAME'] == attraction_name]
    df_attraction = df_attraction.sort_values(by='DATE')

    season_colors = {
        'spring': 'green',
        'summer': 'red',
        'autumn': 'orange',
        'winter': 'blue'
    }

    # Plot setup
    plt.figure(figsize=(10, 6))
    for season, color in season_colors.items():
        season_df = df_attraction[df_attraction['SEASONALITY'] == season]
        plt.plot(season_df['DATE'], season_df['WAIT TIME'], label=season, color=color)

    # Beautifying the plot
    plt.title(f'Waiting Time for {attraction_name} Over Time', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Waiting Time (minutes)', fontsize=12)
    plt.legend(title='Season')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)


# plot_waiting_times(df_15m, 'Bumper Cars')
plot_waiting_times(df_1d.head(1500), 'Bumper Cars')



NameError: name 'df_1d' is not defined